## Custom Data Cleaning
---

## Learning Objectives

Big langauge model is sampling efficient, which implies that if the data fed to the model were full of misspelled words, vulgar by nature ( This is often the case with uncensored data extracted from web forums/chat rooms), contain large volume of other langauge than the target langauge, or were having undesired and mischievous characteristic. One should definately consider creating a pipeline to clean and filter the data.

Since Megatron-LM will sample sentences from documents during training run, we also need to construct a mechanism to find sentence boundary per document.

As is for data deduplication, we are offering one way, there are many other ways, to deduplicate the data on document level, base on a similarity threshold. Why is it important ? Let's take newspaper for example, imagine, a catastrophic event such as tsunami occured in Thailand which claimed  many lives, such event could be reported repeatedly in a great many news articles all over the world. Wouldn't we want to deduplicate the almost identical news articles, with a good similarity measuring mechanism ?

Similarily, when we blend datasets from a variety of sources in order to obtain big data for training big langauge model, we would want to have a way to deduplicate the repeated documents which are present across the collected datasets.


Therefore, the goal of this lab is to provide some basic tools for cleaning and filtering data which should be carefully applied to custom langauge datasets, and preserve the inherit charateristics in the datasets. 

In particular, this notebook covers the following steps :

    1. Langauge Detection. 
    2. Find sentence boundaries.
    3. Deduplicate documents based on similarity score.
Note: The method recommanded in the [Megatron-LM repo, namely LSH](https://github.com/NVIDIA/Megatron-LM/tree/main/tools/openwebtext) will be used for deduplication.

What this notebook will NOT cover :

    - Constructing a black-list to block and filter out inappropriate words/sentences/document.
    - Clean empty lines or empty sentences.
    - Spell-check words and punctuations.
    - Remove sentences/documents with too few tokens.
     and many more customized data cleaning methods which one should consider adding to the data cleaning pipelins.

At the end, there will be a **mini challenge** for hands-on practicing identifying the numbers of duplicated documents as close as you can to the groudtruth number!

---
We will start by installing all the python libraries we need.

In case you encounter problems of installing LSH, here is the fix :

    Install LSH - 

    Follow instruction from [Megatron-LM/tools/openwebtext/README](https://github.com/NVIDIA/Megatron-LM/tree/main/tools/openwebtext) in openwebtext clearning folder 

    Note : In a restricted environment where sudo is not allowed, please follow the below instruction to modify installation.
            
            Call out a terminal as illustrated below.             
   ![call out a terminal ](../../pics/Alt_callout2terminals.JPG)
   
            cd gpubootcamp/ai/Megatron/English/Python/jupyter_notebook/Megatron-LM/tools/openwebtext/
        
            git clone https://github.com/mattilyra/LSH.git
            cd LSH
            pip install -U --user cython>=0.24.1
            open setup.py in an editor and modify as below
   ![modify setup.py line 6](../../pics/modifyLSH_setuppy.JPG)

            python setup.py install --user 

In [ ]:
!pip install ftfy langdetect numpy torch pandas nltk sentencepiece boto3 tqdm regex bs4 htmlmin tldextract sentence-splitter

1. Langauge Detection  

In [ ]:
from langdetect import detect
swe_raw_text='Under fredagsförmiddagen höll polis och räddningstjänst presskonferens tillsammans med en representanter från flygplatsens egna räddningsenhet och Örebro kommun.'
detect(swe_raw_text)

In [ ]:
danish_text='1. januar 2021 var folketallet 5.840.045. Ved den første folketælling i 1735 var der 718.000 danskere.'
detect(danish_text)

In [ ]:
finnish_text='Jokaisella on oikeus vapaasti osallistua yhteiskunnan sivistyselämään, nauttia taiteista sekä päästä osalliseksi tieteen edistyksen mukanaan tuomista eduista.'
detect(finnish_text)

Once we have a way to identify which langauge does this document belong to, we can then filter or remove the documents at will and keep only the selected language(s).

2. Find sentence boundaries - alternative 1 : NLTK

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
text='Detta är ett stycke. Den innehåller flera meningar. “Men varför”, frågar du? Andersson pekas ut som nästa partiledare: “Medlemmarna ska säga sitt”'
print("original doc is :\n ", text)
sents=sent_tokenize(text)
i=0
for sent in sents:
    print("------- sentence {} -------".format(str(i)))    
    print(sent)
    i+=1

Below is the expected outputs :

Observe how NLTK tokenizer sentence per given document :

    ------- sentence 0 -------
    Detta är ett stycke.
    ------- sentence 1 -------
    Den innehåller flera meningar.
    ------- sentence 2 -------
    “Men varför”, frågar du?
    ------- sentence 3 -------
    Andersson pekas ut som nästa partiledare: “Medlemmarna ska säga sitt”

2. Find sentence boundaries - alternative 2 : NLTK + custom function 

In [ ]:
import re
import nltk
from nltk.tokenize import sent_tokenize
def normal_cut_sentence(temp):
    return sent_tokenize(temp)

def cut_sentence_with_quotation_marks(text):
    p = re.compile("“.*?”")
    ls = []
    index = 0
    length = len(text)
    for i in p.finditer(text):
        temp = ''
        start = i.start()
        end = i.end()
        for j in range(index, start):
            temp += text[j]
        if temp != '':
            temp_list = normal_cut_sentence(temp)
            ls += temp_list
        temp = ''
        for k in range(start, end):
            temp += text[k]
        if temp != ' ':
            ls.append(temp)
        index = end
    return ls

In [ ]:
sents=cut_sentence_with_quotation_marks(text)
i=0
for sent in sents:
    print("------- sentence {} -------".format(str(i)))  
    print(sent)
    i+=1

Below is the expected outputs :

Observe how the custom function `cut_sentence_with_quotation_marks` modifies NLTK and adding quotation marks as an additional sentence-splitter :

        ------- sentence 0 -------
        Detta är ett stycke.
        ------- sentence 1 -------
        Den innehåller flera meningar.
        ------- sentence 2 -------
        “Men varför”
        ------- sentence 3 -------
        , frågar du?
        ------- sentence 4 -------
        Andersson pekas ut som nästa partiledare:
        ------- sentence 5 -------
        “Medlemmarna ska säga sitt”
        
There are many ways to split the sentence within a document, the above is just one example.

One can construct other custom function to do sentence splitting with or without NLTK. 

3. Deduplicate documents based on similarity score.

[Local Sensitive Hash](http://snap.stanford.edu/class/cs246-2012/slides/03-lsh.pdf)

First, we create shingles from the document with ngram, then fingerprints were created and Jaccard Similarity measured is used in order to find the top K most similar items given pairs of documents based on an arbitrary threshold.

In [ ]:
import itertools
from lsh import cache, minhash # https://github.com/mattilyra/lsh
from lsh import minhash

# a pure python shingling function that will be used in comparing
# LSH to true Jaccard similarities
def shingles(text, char_ngram=5):
    return set(text[head:head + char_ngram] for head in range(0, len(text) - char_ngram))


def jaccard(set_a, set_b):
    intersection = set_a & set_b
    union = set_a | set_b
    return len(intersection) / len(union)


def candidate_duplicates(document_feed, char_ngram=5, seeds=100, bands=5, hashbytes=4):
    char_ngram = char_ngram
    sims = []
    hasher = minhash.MinHasher(seeds=seeds, char_ngram=char_ngram, hashbytes=hashbytes)
    if seeds % bands != 0:
        raise ValueError('Seeds has to be a multiple of bands. {} % {} != 0'.format(seeds, bands))
    
    lshcache = cache.Cache(num_bands=bands, hasher=hasher)
    for i_line, line in enumerate(document_feed):
        line = line.decode('utf8')
        docid, headline_text = line.split('\t', 1)
        fingerprint = hasher.fingerprint(headline_text.encode('utf8'))
        
        # in addition to storing the fingerpring store the line
        # number and document ID to help analysis later on
        lshcache.add_fingerprint(fingerprint, doc_id=(i_line, docid))

    candidate_pairs = set()
    for b in lshcache.bins:
        for bucket_id in b:
            if len(b[bucket_id]) > 1:
                pairs_ = set(itertools.combinations(b[bucket_id], r=2))
                candidate_pairs.update(pairs_)
    
    return candidate_pairs

We want to verify how good this algorithm is, hence, we will create deduplcates documents from our toy data `extractedNVblogs.txt` obtained in webscaping lab. We will flag the duplicated documents, in the column `duplicate=True`, and this column will serve as the groundtruth for us, we will use this hand-crafted data to verify whether this algorithm will work as expected.

In [ ]:
import pandas as pd
cols=['doc1']
df=pd.read_csv('../../../../dataset/EN/extractedNVblogs.txt',sep='\n', names=cols ,skiprows=1)
df.head()

In [ ]:
import numpy as np

def create_duplicates(df):
    doc2=[]
    duplicate=[]
    n=len(df)
    for i in range(n):
        other_population=[k for k in range(n) if k!=i]
        
        other_idx=np.random.choice(other_population)
        current_idx=np.random.choice([i,other_idx], p=[0.3,0.7])
        if current_idx==i:            
            duplicate.append(True)
        else:
            duplicate.append(False)
        doc2.append(df.iloc[current_idx,0])
    df['index']=df.index
    df['doc2']=doc2
    df['duplicate']=duplicate
    cols=['index','doc1','doc2','duplicate']
    df=df[cols]
    return df

In [ ]:
df2=create_duplicates(df)
df2.tail()

Below is the expected outputs :
    
            
            index 	doc1 	doc2 	duplicate
            65 	65 	This post was updated July 20, 2021 to reflect... 	This post was updated July 20, 2021 to reflect... 	True
            66 	66 	Researchers, developers, and engineers worldwi... 	This post was originally published in August 2... 	False
            67 	67 	Looking to reveal secrets of days past, histor... 	The NVIDIA Deep Learning Institute (DLI) exten... 	False
            68 	68 	Scientists searching the universe for gravitat... 	Robotics researchers from NVIDIA and Universit... 	False
            69 	69 	At GTC ’21, experts presented a variety of tec... 	The NVIDIA Hardware Grant Program helps advanc... 	False

In [ ]:
## this is our groundtruth, count duplicate == True is 31 
df2.duplicate.value_counts()

Below is the expected outputs :
    
    False    45
    True     25
    Name: duplicate, dtype: int64

In [ ]:
df2.columns

In [ ]:
del df
keep_cols_to_write=['index','doc1','doc2']
df3=df2[keep_cols_to_write]
df3.head()

Below is the expected outputs :

            index 	doc1 	doc2
        0 	0 	Deep learning models have been successfully us... 	Deep learning models have been successfully us...
        1 	1 	Breast cancer is the most frequently diagnosed... 	In NVIDIA Clara Train 4.0, we added homomorphi...
        2 	2 	The NVIDIA Deep Learning Institute (DLI) exten... 	The NVIDIA Deep Learning Institute (DLI) exten...
        3 	3 	Engineers, product developers and designers ar... 	Deep learning research requires working at sca...
        4 	4 	Despite substantial progress in natural langua... 	NVIDIA announces our newest release of the CUD...

We could proceed to write the above dataframe to a csv file, however, for mini challenge, we would like to keep determinism. 

In order to preserve determinism, we will load the previously saved df2.csv file so that all attendees have the exact same file.

The `df2.csv` file is provided in this repo.


In [ ]:
## to do this deterministically , let's read in the saved df2.csv file
df2=pd.read_csv('df2.csv', names=['index', 'doc1', 'doc2', 'duplicate'], skiprows=1)
df2.head()

Below is the expected outputs , outputs should look exactly the same as below:

        index 	doc1 	doc2 	duplicate
        0 	0 	Today, NVIDIA announced new pretrained models ... 	Astrophysics researchers have long faced a tra... 	False
        1 	1 	This post was updated July 20, 2021 to reflect... 	This post was updated July 20, 2021 to reflect... 	True
        2 	2 	In part 1 of this series, we introduced new AP... 	Edge computing has been around for a long time... 	False
        3 	3 	The NVIDIA NGC team is hosting a webinar with ... 	The NVIDIA NGC team is hosting a webinar with ... 	True
        4 	4 	NVIDIA announces our newest release of the CUD... 	As an undergraduate student excited about AI f... 	False

In [ ]:
## this is our groundtruth, count duplicate == Truth is 31 
df2.duplicate.value_counts()

Below is the expected outputs , outputs should look exactly the same as below:

            False    42
            True     31
            Name: duplicate, dtype: int64

In [ ]:
!wc -l groundtruth.txt

Below is the expected outputs , outputs should look exactly the same as below:

    73 groundtruth.txt


## Mini-Challenge

Task : 
    Overwrite the below parameters before calling `candidate_duplicates()` function, and rerun the cell block below.
    
    char_ngram= < input_value >
    seeds=< input_value >
    bands=< input_value >
    hashbytes=< input_value >

Pass : Consider yourself pass this mini challenge when you approach the number **31 +/- 3** ! 


Re-run the below cell for experiments in order to get as close as possible to the ground truth = 31 duplicates.
<a id="Rerun_Cell"></a>

In [ ]:
## this is the Re Run Cell 
import itertools
import random
lines = []
with open('groundtruth.txt', 'rb') as fh:
    # read the first 1000 lines into memory so we can compare them
    for line in itertools.islice(fh, 1000):
        lines.append(line.decode('utf8'))
    
    # reset file pointer and do LSH
    fh.seek(0)
    feed = itertools.islice(fh, 1000)

    ## modify the below numbers as input to function candidate_duplicates()
    ## initial value given below, please modify them accordingly to obtain count of number of duplicates to be as close as 31 (= groundtruth)
    char_ngram=13
    seeds=100
    bands=5
    hashbytes=8
    
    candidates = candidate_duplicates(feed, char_ngram=char_ngram, seeds=seeds, bands=bands, hashbytes=hashbytes)

# go over all the candidates comparing their similarities
similarities = []
for ((line_a, docid_a), (line_b, docid_b)) in candidates:
    doc_a, doc_b = lines[line_a], lines[line_b]
    shingles_a = shingles(lines[line_a])
    shingles_b = shingles(lines[line_b])
    
    jaccard_sim = jaccard(shingles_a, shingles_b)
    fingerprint_a = set(hasher.fingerprint(doc_a.encode('utf8')))
    fingerprint_b = set(hasher.fingerprint(doc_b.encode('utf8')))
    minhash_sim = len(fingerprint_a & fingerprint_b) / len(fingerprint_a | fingerprint_b)
    similarities.append((docid_a, docid_b, jaccard_sim, minhash_sim))

for a,b,jsim, msim in random.sample(similarities, k=2 ):
    print("pair of similar sentences with jaccard_sim score:{} and minhash_sim score:{} --- \n".format(str(jsim),str(msim)))
    a=int(a)
    b=int(b)
    text_a=df2.iloc[a,1]
    text_b=df2.iloc[b,2]
    if text_a==text_b:
        print("100% duplicates \n")
    print("text_a:", text_a.split(' ')[:5])
    print("text_b:", text_b.split(' ')[:5])
    print('-----'*10)
    import random

print('\nThere are **{}** candidate duplicates in total\n'.format(len(candidates)))
random.sample(similarities, k=1)

Below is the expected outputs :

A naive run

        pair of similar sentences with jaccard_sim score:0.8197797952482132 and minhash_sim score:0.639344262295082 --- 

        text_a: ['The', 'NVIDIA,', 'Facebook,', 'and', 'TensorFlow']
        text_b: ['Deep', 'learning', '(DL)', 'is', 'the']
        --------------------------------------------------
        pair of similar sentences with jaccard_sim score:0.9133693568066934 and minhash_sim score:0.8867924528301887 --- 

        100% duplicates 

        text_a: ['The', 'first', 'post', 'in', 'this']
        text_b: ['The', 'first', 'post', 'in', 'this']
        --------------------------------------------------

        There are **3** candidate duplicates in total


WOW that is way too LOW !!! We should have 31 duplicates as groundtruth.

<a id="TheChallenge"></a>

Go back and rerun cell <a href="./Lab2-2_SentenceBoundary_and_Deduplicate.ipynb#Rerun_Cell">Jump to ReRun Cell</a>

Solution will be delivered to you at the end of the bootcamp !


In [ ]:
# turn the candidates into a dictionary so we have easy access to
# candidates pairs that were found
candidates_dict = {(line_a, line_b): (docid_a, docid_b) for ((line_a, docid_a), (line_b, docid_b)) in candidates}
found = 0
for i in range(len(lines)):
    for j in range(i+1, len(lines)):
        if sims_all[i, j] >= .9:
            # documents i and j have an actual Jaccard similarity >= 90%
            found += ((i, j) in candidates_dict or (j, i) in candidates_dict)

print('Out of {} pairs with similarity >= 90% {} were found, that\'s {:.1%}'.format((sims_all >= .9).sum(), found, found / (sims_all >= .9).sum()))


In [ ]:
num_candidates = []
bands = [2, 5, 10, 20]
for num_bands in bands:
    with open('groundtruth.txt', 'rb') as fh:
        feed = itertools.islice(fh, 1000)
        candidates = candidate_duplicates(feed, char_ngram=5, seeds=100, bands=num_bands, hashbytes=4)
        num_candidates.append(len(candidates))

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
fig, ax = plt.subplots(figsize=(8, 6))
plt.bar(bands, num_candidates, align='center');
plt.title('Number of candidate duplicate pairs found by LSH using 100 minhash fingerprint.');
plt.xlabel('Number of bands');
plt.ylabel('Number of candidate duplicates');
plt.xticks(bands, bands);


--- 
## Links and Resources
Don't forget to check out additional resources such as [Language Detect](https://github.com/Mimino666/langdetect), [NLTK Sentence Tokenizer](https://www.nltk.org/api/nltk.tokenize.html) and [Local Sensitive Hashing](http://snap.stanford.edu/class/cs246-2012/slides/03-lsh.pdf).

-----
## <p style="text-align:center;border:3px; padding: 1em"> <a href=../../../../Start_Here.ipynb>HOME</a>&nbsp; &nbsp; &nbsp; <a href=../../../Lab2-3_train_own_GPT2BPETokenizer.ipynb>NEXT</a></p>


-----


## Licensing 

This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0). 